In [1]:
import torch
import torchaudio
from IPython.display import clear_output
import random
import gc

In [2]:
torchaudio.set_audio_backend("soundfile")

/tmp/ipykernel_676814/1822137616.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks/swishnet')
# import sys
# sys.path.append(os.getcwd())

In [4]:
sampling_rate = 16000
suffix = "_16K_random_cut"

In [5]:
def collect_wav_files(base_dir):
    collected = []
    for root, _, files in os.walk(base_dir):  # 모든 하위 디렉토리 탐색
        for fname in files:
            if fname.lower().endswith(".wav"):  # 확장자 필터
                full_path = os.path.join(root, fname)
                collected.append(full_path)
    return collected


In [6]:
unpreprocessed_car_dir = "./car"
unpreprocessed_music_dir = "./music"
unpreprocessed_noise_dir = "./noise"
unpreprocessed_speech_dir = "./speech"

In [7]:
unpreprocessed_car_list = collect_wav_files(unpreprocessed_car_dir)
unpreprocessed_music_list = collect_wav_files(unpreprocessed_music_dir)
unpreprocessed_noise_list = collect_wav_files(unpreprocessed_noise_dir)
unpreprocessed_speech_list = collect_wav_files(unpreprocessed_speech_dir)

In [8]:
def get_wav_info_torchaudio(filepath):
    info = torchaudio.info(filepath)
    
    print(f"파일 경로: {filepath}")
    print(f"샘플링 레이트 (Hz): {info.sample_rate}")
    print(f"채널 수: {info.num_channels}")
    print(f"총 프레임 수: {info.num_frames}")
    print(f"재생 시간 (초): {info.num_frames / info.sample_rate:.2f}")
    print(f"비트 깊이: {info.bits_per_sample}")

In [9]:
print(unpreprocessed_car_list[0])
get_wav_info_torchaudio(unpreprocessed_car_list[0])


./car/vehicle_vehicle/bus/Bus_Driving_Sound_1_HOUR_650.wav
파일 경로: ./car/vehicle_vehicle/bus/Bus_Driving_Sound_1_HOUR_650.wav
샘플링 레이트 (Hz): 44100
채널 수: 2
총 프레임 수: 220500
재생 시간 (초): 5.00
비트 깊이: 16


In [10]:
resamplers = {}  # script# 리샘플러 캐시 (원본 sample_rate -> 변환기)

def preprocess(preprocessed_data_list):
    data_list = []  # 최종적으로 반환할 세그먼트 리스트

    for path in preprocessed_data_list:
        try:
            print(f" Processing: {path}")
            waveform, sample_rate = torchaudio.load(path)  # 오디오 파일 로딩

            # Resample if needed
            if sample_rate != sampling_rate:
                if sample_rate not in resamplers:
                    resamplers[sample_rate] = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=sampling_rate)
                waveform = resamplers[sample_rate](waveform)

            # Mono conversion
            if waveform.shape[0] > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
            waveform = waveform.squeeze(0)

            total_length = waveform.shape[0]  # 전체 길이 (샘플 수)
            start = 0  # 시작 위치 초기화

            # 랜덤 길이로 중복 없이 자르기
            while start + 16000 <= total_length:  # 최소 1초라도 남아있을 때까지 반복
                try:
                    max_possible = min(48000, total_length - start)  # 현재 위치에서 가능한 최대 길이
                    cutsize = random.randint(16000, max_possible)  # 랜덤한 자르기 길이 (1~3초)
                    
                    segment = waveform[start:start + cutsize]  # 자르기
                    data_list.append(segment)  # 결과 리스트에 추가
                    
                    start += cutsize  # 시작 위치를 자른 길이만큼 이동
                except Exception as seg_error:
                    print(f"⚠️ Error cutting segment from {path}: {seg_error}")
                    break  # segment 단위 오류 발생 시 중단

            del waveform, segment  # 메모리 정리
            gc.collect()

        except Exception as e:
            print(f" Failed to process {path}: {e}")
            continue

        clear_output(wait=True)  # Jupyter 환경에서는 출력 깔끔하게
    return data_list


In [11]:
# car_list = preprocess(unpreprocessed_car_list)
# print(len(car_list))
# torch.save(car_list, f"../unsplited/car/car_data{suffix}.pt")

In [12]:
# music_list = preprocess(unpreprocessed_music_list)
# print(len(music_list))
# torch.save(music_list, f"../unsplited/music/music_data{suffix}.pt")

In [13]:
# noise_list = preprocess(unpreprocessed_noise_list)
# print(len(noise_list))
# torch.save(noise_list, f"../unsplited/noise/noise_data{suffix}.pt")

In [ ]:
# speech_list = preprocess(unpreprocessed_speech_list)
# print(len(speech_list))
# torch.save(speech_list, f"../unsplited/speech/speech_data{suffix}.pt")

108820
